# Geolocalizador

Autores: Sofía Barreto Ojeda Néstor Felipe Bobadilla, David Gamba

El objetivo de este proyecto es hacer una visualizacion de los eventos bélicos en territorio ucraniano y atribuidos al hostigamiento ruso. Este notebook incluye tres secciones:

**1. Descarga de Datos.**

**2. Analisis Exploratorio de Datos.**

**3. Visualización de Datos.**

**4. Siguientes pasos.**

Vamos a usar las librerias: 
- `urllib` para extraer la informacion, 
- `pandas`, para tabular la informacion encontrada.
- `geoplotlib` y `plotly`, para generar el mapa de Ucrania y visualizar la golocalizacion de los bombardeos.

-----

Antes de iniciar, importamos las librerías que vamos a usar en este notebook:

In [122]:
# descargar datos
from urllib.request import urlopen

#json
import json

# tabular
import pandas as pd

# maps
import plotly.express as px

----

## 1. Descarga de Datos

Los detalles de los eventos se obtienen de la base de datos proviene del grupo de periodismo investigativo [Bellingcat](https://en.wikipedia.org/wiki/Bellingcat). Esta base de datos que muestra fecha, hora, ubicación, y coordenadas de cada evento.

La base de datos esta disponible en el siguiente link, al que llamamos `url`:

In [123]:
url = 'https://ukraine.bellingcat.com/ukraine-server/api/ukraine/export_events/deeprows'

El siguiente paso es guardar la informacion de nuestro url en la variable `data`. Dado que los datos están almacenados en formato json, podemos usar la función `json.load` para almacenar nuestros datos en la variable `datos`.

Así, almacenamos la información en `datos`:

In [124]:
with urlopen(url) as response:
    datos = json.load(response)

Podemos revisar el numero de eventos registrados con `len`, de donde se puede ver que hay 432 entradas.

In [125]:
len(datos)


445

Ya que tenemos la información almacedanada en la variable `datos`, podemos proceder a hacer un breve análisis exploratorio de datos.

## 2. Análisis Exploratorio de Datos

Como primer paso, podemos revisar el tipo de información obtenida de la base de datos. Dado que la información está contenida en un archivo de formato json, podemos extraer las llaves del primer elemento de la variable `datos` para saber el tipo de datos con los que contamos:

In [126]:
columns = (datos[0].keys()) # muestra las llaves de la primera entrada de la variable datos
print(f"{len(columns)} columnas.\n{columns}")

9 columnas.
dict_keys(['associations', 'sources', 'id', 'description', 'date', 'time', 'location', 'latitude', 'longitude'])


De aqui podemos ver que tenemos 9 diferentes columnas:
- `associations` 
- `sources` 
- `id` 
- `description` 
- `date` 
- `time`
- `location` 
- `latitude`
- `longitude`

Ahora que tenemos una mejor idea del tipo de información almacenada en `datos`, podemos usas la libreria [pandas](https://pandas.pydata.org/docs/index.html) para usar `DataFrames` en lugar de formato json.

### 2.1 Json a DataFrame

Para convertir la información de json a DataFrames, podemos usar la función `json_normalize` disponible en la libreria pandas. Por ejemplo, para la primera entrada podemos convertir json a `DataFrame` con:

In [127]:

df1 = pd.json_normalize(datos) # convierte json a dataframe, primera entrada
df1

,associations,sources,id,description,date,time,location,latitude,longitude
0,"[asc1, asc11]","[src_exists_CIV0001_1, src_exists_CIV0001_2, s...",CIV0001,"Apartment block hit. Crater is very large, pos...",02/24/2022,00:00,"Chuhuiv, south of Kharkiv",49.85005,36.659031
1,"[asc8, asc21]","[src_exists_CIV0002_1, src_exists_CIV0002_2, s...",CIV0002,Civillians hit by what appears to have been ar...,02/24/2022,00:00,Uman,48.748564,30.218515
2,[],[srcHidden],CIV0003,"Explosion in central Kyiv, nothing further yet.",02/24/2022,00:00,Kyiv,50.470055,30.527381
3,"[asc4, asc14]","[src_exists_CIV0004_1, src_exists_CIV0004_2, s...",CIV0004,"Explosion in central Kyiv, nothing further yet.",02/24/2022,00:00,Vuhledar,47.775537,37.239601
4,"[asc1, asc11]","[src_exists_CIV0008_1, src_exists_CIV0008_2]",CIV0008,Civillian homes hit by artillery.,02/24/2022,00:00,Mariupol,47.116736,37.684468
...,...,...,...,...,...,...,...,...,...
440,"[asc8, asc11]",[src_exists_CIV0907_1],CIV0907,Munition remnant embedded in pavement of parki...,03/24/2022,00:00,Kharkiv,50.020498,36.239983
441,"[asc9, asc11]",[src_exists_CIV0908_1],CIV0908,"Menorah statue damaged, allegedly by artillery.",03/26/2022,00:00,Kharkiv,49.932311,36.439195
442,"[asc5, asc11]",[src_exists_CIV0911_1],CIV0911,School destroyed by explosion.,03/04/2022,00:00,Izyum,49.342847,37.527481
443,"[asc9, asc11]",[srcHidden],CIV0916,Destruction of a church.,05/09/2022,00:00,Donetsk,48.992607,37.443586


Aquí llamamos `df1` a la variable que almacena una DataFrame para la primera entrada.

### 2.2. Suprimir información no necesaria

Adicionalmente, dado que nuestro objetivo es visualizar eventos (geolocalizador), de la información obtenida podemos suprimir las columnas `associations`, `sources`, `id`, y `time`. Para borras estas filas, podemos usar `.drop()`:

In [128]:
df = df1.drop(columns=['associations', 'sources', 'id', 'time'])

Verificamos que los datos pertenecientes a nuestro DataFrame, sean depurados.
Antes de continuar, también debemos verificar que los valores de la columna `latitude` and `longitude` son de typo `float`. 

In [129]:
df

,description,date,location,latitude,longitude
0,"Apartment block hit. Crater is very large, pos...",02/24/2022,"Chuhuiv, south of Kharkiv",49.85005,36.659031
1,Civillians hit by what appears to have been ar...,02/24/2022,Uman,48.748564,30.218515
2,"Explosion in central Kyiv, nothing further yet.",02/24/2022,Kyiv,50.470055,30.527381
3,"Explosion in central Kyiv, nothing further yet.",02/24/2022,Vuhledar,47.775537,37.239601
4,Civillian homes hit by artillery.,02/24/2022,Mariupol,47.116736,37.684468
...,...,...,...,...,...
440,Munition remnant embedded in pavement of parki...,03/24/2022,Kharkiv,50.020498,36.239983
441,"Menorah statue damaged, allegedly by artillery.",03/26/2022,Kharkiv,49.932311,36.439195
442,School destroyed by explosion.,03/04/2022,Izyum,49.342847,37.527481
443,Destruction of a church.,05/09/2022,Donetsk,48.992607,37.443586


Como se puede verificar en la siguiente celda, las columnas `latitude` and `longitude` son de typo `str`.

In [130]:
type(df['latitude'][0])
type(df['longitude'][0])

str

### 2.3. Convertir tipos de variables

Para convertirlas a float hacemos:

In [131]:
#df.astype({'latitude': 'float', 'longitude':'float'}).dtypes
df["latitude"]= df["latitude"].astype(float)
type(df["latitude"][0])
df['longitude']= df['longitude'].astype(float)
type(df['longitude'][0])


numpy.float64

De aquí también podemos ver que la columna `date` requiere de formato de fecha. Para asignar el formato apropiado hacemos:

In [132]:
df['date'] = pd.to_datetime(df['date'])

De forma que tenemos una DataFrame con 5 columnas:

In [133]:
df

,description,date,location,latitude,longitude
0,"Apartment block hit. Crater is very large, pos...",2022-02-24,"Chuhuiv, south of Kharkiv",49.850050,36.659031
1,Civillians hit by what appears to have been ar...,2022-02-24,Uman,48.748564,30.218515
2,"Explosion in central Kyiv, nothing further yet.",2022-02-24,Kyiv,50.470055,30.527381
3,"Explosion in central Kyiv, nothing further yet.",2022-02-24,Vuhledar,47.775537,37.239601
4,Civillian homes hit by artillery.,2022-02-24,Mariupol,47.116736,37.684468
...,...,...,...,...,...
440,Munition remnant embedded in pavement of parki...,2022-03-24,Kharkiv,50.020498,36.239983
441,"Menorah statue damaged, allegedly by artillery.",2022-03-26,Kharkiv,49.932311,36.439195
442,School destroyed by explosion.,2022-03-04,Izyum,49.342847,37.527481
443,Destruction of a church.,2022-05-09,Donetsk,48.992607,37.443586


donde podemos ver que la fecha contenida en la columna `date` ahora tiene formato de `año-mes-día`.

De la DataFrame obtenida, podemos identificar información clave para construir nuestro geolocalizador. Las dos columnas de datos más importantes son las de `latitude` and `longitud`, las cuales proveen información geográfica para graficar en el mapa.

## 3. Visualización de Datos

Para visualizar los datos de latitud y longitud podemos hacer uso de la librería [plotly express](https://plotly.com/python/plotly-express/).

Especificamente, con [scatter_mapbox](https://plotly.com/python/scattermapbox/) podemos crear un gráfico de dispersión superpuesto sobre el mapa de Ucrania. Cada fila de una data frame es representada por un punto o símbolo (marcador).

Por ejemplo, para visualizar el primer evento de agresión registrado como parte de la estrategia Rusa en el conflicto con Ucrania, podemos usar `px.scatter_mapbox`.

```Python
px.scatter_mapbox(lon =,          # datos de longitud
                  lat =,          # datos de latitud
                  mapbox_style =, # estilo del mapbox
                  zoom =,         # zoom
                  size =          # tamaño del marcador
                 )
```

In [139]:
px.scatter_mapbox(lon=df['longitude'], lat=df['latitude'], mapbox_style='open-street-map', zoom=3, size=df.index+100)

De esta forma podemos visualizar de forma interactiva la información disponible en nuestra base da datos.

---

Para hacer de una forma mas organizada la interpretación del mapa, creamos funciones para que al usuario le sea facil el filtrar la información que este considere relevante.

Filtrado por fecha

Para que sea posible para el usuario filtrar los datos para visualizar los acontecimientos en una fecha específica, se crea un DataFrame aparte que ilustre los datos filtrados, para esto la librería pandas tiene un método que nos permite darle al programa un rango de fechas.  - df.loc() - El cual delimitará nuestros datos


In [165]:
def visualizar_por_fecha_especifica():

        z=input("Responda Si o No\n ¿Le interesa saber los bombardeos ocurridos en una fecha específica")
        if z=="Si" or z=="si":

                a=input("ingrese la fecha en formato M/D/A")
                filteredfechaesp_df = df.loc[df["date"].between(a, a)]
        
                b=px.scatter_mapbox(lon=filteredfechaesp_df['longitude'], lat=filteredfechaesp_df['latitude'], mapbox_style='open-street-map', zoom=3, size=filteredfechaesp_df.index+100)
                
                return b
        elif z=="No" or z=="no":
                j=input("ingrese la fecha desde donde quiere empezar a analizar, en formato M/D/A")
                h =input("Ingrese la fecha limite hasta donde quiere usted hacer la visualización")
                filteredfechaesp_df = df.loc[df["date"].between(j, h)]
        
                b=px.scatter_mapbox(lon=filteredfechaesp_df['longitude'], lat=filteredfechaesp_df['latitude'], mapbox_style='open-street-map', zoom=3, size=filteredfechaesp_df.index+100)
                return b
        




        

In [222]:
visualizar_por_fecha_especifica()

Verficar dato por Ubicación

Para que el usuario pueda conocer si alguna ubicación de su interés ha sido damnificada, mediante el metos - isin()- de la librería pandas, este podra observar una lista de booleanos el cual le confirmará si su dato buscado se encuentra dentro de la base de datos.

In [228]:
def Buscara_disponibilidad_de_dato_por_ubicacion(Ubicacion):
    a=df.location.isin([Ubicacion])
    return a
    


In [230]:
Buscara_disponibilidad_de_dato_por_ubicacion("Uman")

0      False
1       True
2      False
3      False
4      False
       ...  
440    False
441    False
442    False
443    False
444    False
Name: location, Length: 445, dtype: bool

Filtrar Por Ubicación

Para este tipo de filtro se ha de hacer una mascara de Booleanos (vector de Booleanos), el cual retornará de la base de datos. Esta "mascara" le informará al programa, que datos buscados, se encuentran y cuales no, en la base de datos.Por lo tanto al usar esta mascara dentro del Data frame, nos retornará, la base con las filas que si contengan la ubicación buscada. 

Por consiguiente se le intuye al programa que gráfique la información contenida en el Data Frame filtrado.

In [223]:
def visualizar_zonas_ciudad():
    n=input("Ingrese la urbanizacion Ukraniana de su interés, ejemplo Kyiv.\n Ingrese el nombre: ")
    
    df_mask=df['location']==n
    filteredbycity_df = df[df_mask]
    
        
    plot=px.scatter_mapbox(lon=filteredbycity_df['longitude'], lat=filteredbycity_df['latitude'], mapbox_style='open-street-map', zoom=3, size=filteredbycity_df.index+100)
    return plot
   

In [224]:
visualizar_zonas_ciudad()